# Create a word cloud

In this notebook we give a high level summary of a large English text.

We do that in the form of a wordcloud.

### Here is the text in PDF

In [ ]:
from IPython.display import HTML
HTML('<iframe src=http://maartenmarx.nl/teaching/CollectieveIntelligentie/Data/thesis-main_final.pdf width=700 height=700></iframe>')

In [1]:
# turn the pdf into a text file 

# Files staan op http://maartenmarx.nl/teaching/CollectieveIntelligentie/Data/

!pdftotext -layout  -enc UTF-8 ../Data/thesis-main_final.pdf 

/bin/sh: pdftotext: command not found


In [ ]:
# Hoe groot is de file en hoeveel woorden staan erin
!ls -lh ../Data/thesis-main_final.*; wc -w ../Data/thesis-main_final.txt

In [ ]:
# Hoe ziet de tekst er uit?
!head -30 ../Data/thesis-main_final.txt

In [ ]:
import nltk
import re
from collections import Counter
from pattern.en import  lemma  # also exists for Dutch
import pattern.nl #  import lemma

In [ ]:
# After a lot of hassle we discovered how to fix the encoding problem....

text= open('../Data/thesis-main_final.txt').read().decode('utf-8')


In [ ]:
# Tokenize
TextTokens= nltk.word_tokenize(text)

In [ ]:
# Check
len(TextTokens),TextTokens[:20]

# Text normalization

1. remove interpunction tokens
2. lowercase
3. lemmatize
4. remove short words
5. remove stopwords

In [ ]:
# Inspect the stopwords
stopwords= set(nltk.corpus.stopwords.words('english')) # we turn it into a set because of speed
len(stopwords), list(stopwords)[:10]

In [ ]:
TextTokens[:20]

In [ ]:
# Text normalization Try out

[lemma(s.lower()) for s in TextTokens[:20] 
     if not  re.match(r'^\W+$',s) and 
        len(s) >3 and
         not s in stopwords
]

In [ ]:
# Do it for all 
NormalizedTextTokens= [lemma(s.lower()) for s in TextTokens
     if not  re.match(r'^\W+$',s) and 
        len(s) >3 and
         not s in stopwords
]

# Do some repairs 
# try re.sub(u'\ufb01','fi', u'de\ufb01nition')
# with lemmatization we get new 3 letter words
NormalizedTextTokens=[re.sub(u'\ufb01','fi',w) for w in NormalizedTextTokens if len(w) >3 ]

# See the reduction in number of tokens and number of types
for x in [NormalizedTextTokens,TextTokens]:
    print len(x), len(set(x))


In [ ]:
# Now count

WordCloud= Counter(NormalizedTextTokens)
WordCloud.most_common(50)

In [ ]:
# Change to Wordle format: see http://www.wordle.net/advanced
# remove/repair "wrong" words

for p in WordCloud.most_common(51):
    if not p[0]=='defin':
        print "%s:%s" % (p)

# Tadah: Make a wordcloud

I pasted the above into <http://www.wordle.net/advanced> and got 

<img src='../Data/thesis-main_final.png'/>

# Nu een NL voorbeeld

In [ ]:
!wget http://stichtinggezondheid.nl/wp-content/uploads/2013/09/Van-Partners-naar-Ouders-PDF-eBook-Oei-ik-groei.pdf

In [ ]:
!pdftotext -layout  -enc UTF-8 Van-Partners-naar-Ouders-PDF-eBook-Oei-ik-groei.pdf

In [ ]:
# Inspect the stopwords
stopwords= set(nltk.corpus.stopwords.words('dutch')) # we turn it into a set because of speed
len(stopwords), list(stopwords)[:10]

In [ ]:
text= open('Van-Partners-naar-Ouders-PDF-eBook-Oei-ik-groei.txt').read().decode('utf-8')
TextTokens= nltk.word_tokenize(text)
# Do it for all 
NormalizedTextTokens= [ (s.lower()) for s in TextTokens
     if not  re.match(r'^\W+$',s) and 
        len(s) >3 
         and         not s in stopwords
]

# Do some repairs 
# try re.sub(u'\ufb01','fi', u'de\ufb01nition')
# with lemmatization we get new 3 letter words
NormalizedTextTokens=[re.sub(u'\ufb01','fi',w) for w in NormalizedTextTokens if len(w) >3 ]

# See the reduction in number of tokens and number of types
for x in [NormalizedTextTokens,TextTokens]:
    print len(x), len(set(x))


In [ ]:
WordCloud= Counter(NormalizedTextTokens)
WordCloud.most_common(50)

# Verbeteringen

* lemmatiseer
* pak alleen de zelfstandige naamwoorden, de werkwoorden in de lijst hierboven zegen niet veel, en er zitten toch ook nog veel "stopwoorden" tussen die je dan ook wel kwijt bent.
* Beide kan met de `pattern.nl` module.

### Zelf doen
* Neem een wat langere Nederlandse tekst, bijvoorbeeld een (liefst dik) boek dat je goed kent, en doe hetzelfde. 
* Wat vind je, geeft je woordenwolk een mooi beeld van dat boek?

# Data Science Project

* Verbeter de woorden in de wolken:
    * gooi "domein-specifieke" stopwoorden eruit: _chapter, section, page,..._
    * Voeg bigrams/ colocaties toe _"computer science", "set theory", "brain damage", etc_
* Integreer de code met een download en run code over een heel corpus van een hoogleraar.
    * Download alle proefschriften van PhDs van die hoogleraar via uva dare
    * draai de software erover heen
    * Maak een uniforme verzameling wolken: allicht geordend in de tijd
    * Maak ook een samenvatting van alle proefschriften bij elkaar.
    * Maak ook een tijdsdiagram: waarbij je "trends" in die proefschriften van die hoogleraar aangeeft.

* Doe dit voor 
    * Dick Swaab
    * JFAK van Benthem
    * Maarten de Rijke
    * Iemand uit een totaal ander vakgebied 


* Hieronder een voorbeeld met Dick Swaab.
    * Werkt perfect: je zoekt op een promotor, en download alles wat daaronder zit.
    * in `Swaab/dare.uva.nl/document/2` zitten alle pdfs.
        * 21 proefschriften, 271 files, 532 Mb in totaal.
    * Elk proefschrift lijkt er dubbel in te zitten omdat het er zowel als geheel als in de afzonderlijke hoofstukken in zit. 
    * Jammer genoeg zitten de proefschriften niet in aparte folders.
    * Maar via de search-pages zijn de files wel weer aan de metadata te hangen. 

In [ ]:
%mkdir Swaab
%cd Swaab

In [ ]:
# -r recursief. -np "no-parents" (ga alleen naar beneden met ophalen, niet naar boven). -q geef geen output
!wget -r -np -q  "http://dare.uva.nl/search?field1=promotoren+copromotoren&value1=swaab&join=and&field2=authors+editors&value2=+&smode=advanced&publicatiejaar-min=&publicatiejaar-max=&documenttype=&documenttype-join=or"